## Abstract

This notebook implements a **Continuous-Variable Kerr Quantum Reservoir Computing (CV-Kerr QRC)** model for short-term electricity grid load forecasting using half-hourly UK demand data and exogenous drivers (e.g., temperature and calendar/seasonal structure).

The Kerr nonlinearity introduces a richer reservoir dynamical regime relative to standard CV-QRC, enabling more expressive nonlinear feature expansion prior to a lightweight classical readout. Results are evaluated under a controlled protocol aligned with classical baselines and the companion QRC notebook to support fair benchmarking of quantum-enhanced time-series prediction.


## Authorship and Project Credit

This notebook is derived from a collaborative research project conducted as part of QUAC 605.

Contributing authors:

* Dominic Kosnica
* Sennai Kahssai
* Dawinder Bansrow
* Michael Liu

All collaborators contributed to data processing, modeling, and analysis. This notebook represents a curated and extended version of the original group project for research portfolio purposes.


## Relationship to Classical and QRC Benchmark Notebooks

This notebook is part of a controlled classical–quantum benchmarking series for electricity grid load forecasting:

1. **Classical Baseline (ML-AI repo):**
   *Classical_Baseline_Model_for_Electricity_Grid_Load_Forecasting.ipynb*
   Establishes naive persistence, ridge regression, and random forest benchmarks.

2. **Quantum Reservoir Benchmark (Quantum repo):**
   *Hybrid_Quantum_Classical_Reservoir_Computing_for_Electricity_Grid_Load_Forecasting.ipynb*
   Implements a quantum reservoir approach and establishes the quantum baseline for comparison.

The present notebook extends the quantum reservoir approach by introducing **Kerr nonlinearity** in a continuous-variable reservoir. This allows direct evaluation of whether Kerr-driven dynamics improve forecasting accuracy or efficiency under the same dataset, preprocessing, and evaluation protocol.


## Why CV-Kerr QRC

Standard (linear or weakly nonlinear) reservoirs can be limited in their ability to generate sufficiently expressive nonlinear temporal embeddings from multivariate inputs. A Kerr element introduces a controllable nonlinearity into the continuous-variable dynamics, potentially improving reservoir richness and memory properties while keeping the training procedure lightweight (training only the classical readout).

This notebook evaluates CV-Kerr QRC as a principled architectural upgrade within the same forecasting task, enabling controlled comparison against both classical ML baselines and the non-Kerr quantum reservoir configuration.


## Experimental Protocol

* **Task:** next-step (30-minute ahead) electricity demand forecasting
* **Data:** half-hourly UK demand time series spanning a full year, with exogenous variables (e.g., temperature) and calendar features
* **Splits:** chronological train/validation/test split (no leakage)
* **Baselines:** naive persistence, ridge regression, random forest
* **Quantum models:** QRC baseline and CV-Kerr QRC variant
* **Metrics:** MAE/RMSE (aligned with the project report and baseline notebook)

This protocol is designed to ensure scientific comparability across classical and quantum models.


## Limitations

This study evaluates CV-Kerr QRC primarily in simulation, which may not capture hardware effects such as noise, loss, and finite squeezing in real photonic systems. In addition, simulation cost can grow rapidly with reservoir size and time horizon, limiting scaling experiments in a notebook setting.

Accordingly, conclusions should be interpreted as benchmarking evidence under controlled simulation assumptions and a foundation for future hardware-based evaluation.


### Computational Constraints and Hilbert Space Scaling

An important practical limitation of the present study arises from the rapid growth of the continuous-variable Hilbert space. CV quantum systems scale exponentially with the number of modes and photon-number cutoff, leading to a steep increase in simulation cost as reservoir size and nonlinearity are increased. As a result, the reservoir dimensionality and Kerr strength explored here were necessarily constrained by available computational resources.

This limitation implies that the full dynamical richness of CV-Kerr reservoirs could not be exhaustively explored. Larger reservoirs and higher cutoffs may enable more expressive temporal embeddings and improved memory capacity, but were infeasible within a notebook-scale simulation environment. Consequently, the reported performance should be interpreted as a **lower bound** on the potential of CV-Kerr QRC rather than a definitive performance ceiling.

This computational bottleneck highlights a key motivation for hardware-based continuous-variable quantum computing: physical photonic systems naturally evolve in high-dimensional Hilbert spaces without incurring the exponential simulation cost faced on classical hardware. As such, experimental CV quantum reservoirs may access dynamical regimes that are currently inaccessible to classical simulation.


## Discussion

The CV-Kerr Quantum Reservoir Computing (CV-Kerr QRC) results should be interpreted in the context of strong classical and quantum baselines. Classical Random Forest models remain highly competitive for electricity load forecasting due to their ability to capture nonlinear weather–time interactions and complex demand dynamics. In our classical benchmark, Random Forest achieved the lowest test error among classical models, establishing a stringent reference point for any quantum approach. 

The motivation for introducing quantum reservoirs is not to replace classical models immediately, but to explore alternative learning paradigms that may scale more favorably as model complexity increases. Classical deep models often require large numbers of trainable parameters and repeated optimization across many layers to capture nonlinear temporal structure. By contrast, reservoir computing fixes the internal dynamics and trains only a lightweight readout layer. In the quantum setting, the reservoir’s natural unitary evolution can generate a rich nonlinear feature map without deep parameter optimization. 

The CV-Kerr architecture is particularly attractive because the Kerr nonlinearity enriches reservoir dynamics beyond linear or weakly nonlinear regimes. This enables more expressive temporal embeddings using a small number of quantum modes, which is critical under near-term hardware constraints. Continuous-variable systems naturally operate on amplitudes and phases, matching the structure of electricity demand signals and avoiding heavy discretization or large qubit registers. 

While near-term quantum reservoirs may not yet outperform optimized classical ensembles, they demonstrate a fundamentally different path to nonlinear representation learning. As hardware improves, quantum reservoirs may offer competitive expressivity with reduced training overhead, particularly in large-scale temporal modeling tasks.


## Future Directions

Several extensions can strengthen and scale the CV-Kerr QRC framework.

**Transition to photonic hardware.**
Current experiments rely on simulation, which does not fully capture noise, loss, and finite squeezing present in real continuous-variable systems. Implementing CV-Kerr QRC on photonic platforms would allow evaluation under realistic operating conditions and provide a clearer assessment of near-term feasibility. 

**Hybrid quantum–attention architectures.**
Quantum reservoirs can be combined with lightweight attention mechanisms, where the reservoir generates nonlinear temporal features and an attention layer performs temporal weighting. This hybrid design may improve performance on complex load patterns without requiring deep reservoirs. 

**Long-horizon forecasting.**
The current study focuses on short-term prediction. Extending to longer horizons may require deeper reservoirs or quantum recurrent structures to improve long-range temporal memory. 

**Fault-tolerant scaling.**
In the longer term, fault-tolerant continuous-variable quantum computing could support larger and more stable reservoirs, enabling scaling beyond NISQ limitations and potentially strengthening theoretical guarantees on expressivity and quantum advantage. 

Together, these directions position CV-Kerr QRC as a promising component of future hybrid quantum–classical forecasting systems.


## Reproducibility

The notebook is organized to support deterministic replication of preprocessing, model construction, and evaluation. The CV-Kerr QRC workflow mirrors the classical baseline and companion QRC notebook to preserve controlled benchmarking.

For clean publication presentation, dependency installation is consolidated into a single optional setup cell and notebook outputs are kept minimal and interpretable.


In [ ]:
## Environment Setup (Optional)

If running locally, it is recommended to use a virtual environment or conda environment.
If running in a fresh notebook runtime (e.g., Colab), install dependencies using the cell below.

```python
# Optional: install dependencies in a fresh runtime (e.g., Colab)
# !pip install -q pennylane strawberryfields pandas numpy scikit-learn matplotlib
```

```python
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
```


In [38]:
from google.colab import files

uploaded = files.upload()

filename = list(uploaded.keys())[0]
print(f"Loaded file: {filename}")

# Read
df = pd.read_csv(filename)

# Peek
df.head()


Saving QUAC 605 Project Data_processed.xlsx - 3 month_processed.csv to QUAC 605 Project Data_processed.xlsx - 3 month_processed.csv
Loaded file: QUAC 605 Project Data_processed.xlsx - 3 month_processed.csv


,SETTLEMENT_PERIOD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_SOLAR_GENERATION,PUMP_STORAGE_PUMPING,SCOTTISH_TRANSFER,NEMO_FLOW,ELECLINK_FLOW,Temperature,period_sin,period_cos
0,1,19918,5405,0,859,6353,-558,0,10.5,0.130526,0.991445
1,2,20167,5358,0,735,6333,-564,0,10.8,0.258819,0.965926
2,3,20328,5237,0,691,6380,-205,0,11.1,0.382683,0.923880
3,4,19460,5115,0,931,6250,-193,0,11.1,0.500000,0.866025
4,5,18654,5080,0,854,6142,-622,0,11.1,0.608761,0.793353


In [39]:

# feature column definition (9 total)

feature_cols = [
    "EMBEDDED_WIND_GENERATION",
    "EMBEDDED_SOLAR_GENERATION",
    "PUMP_STORAGE_PUMPING",
    "SCOTTISH_TRANSFER",
    "NEMO_FLOW",
    "ELECLINK_FLOW",
    "Temperature",
    "period_sin",
    "period_cos",
]

target_col = "ENGLAND_WALES_DEMAND"

# Extract input matrix X_raw and target y_raw

X_raw = df[feature_cols].values.astype(float)
y_raw = df[target_col].values.astype(float)

print("X_raw shape:", X_raw.shape)
print("y_raw shape:", y_raw.shape)

# Normalize X and y for CV encoding

X_scaler = StandardScaler()
X = X_scaler.fit_transform(X_raw)

y_scaler = StandardScaler()
y = y_scaler.fit_transform(y_raw.reshape(-1, 1)).ravel()

print("X normalized shape:", X.shape)
print("y normalized shape:", y.shape)


X_raw shape: (4318, 9)
y_raw shape: (4318,)
X normalized shape: (4318, 9)
y normalized shape: (4318,)


In [40]:
# Define mode counts
n_modes_list = [4, 5, 6]

# Number of input features (9 total)
n_features = X.shape[1]
print("n_features:", n_features)

# mode count
n_modes_list = [6]

# Random generator (weights, reservoir phases, beam splitter angles, projections...etc)  #reproducibility
rng = np.random.default_rng(seed=42)

# projection matrix Dictionary (store)
W_enc = {}

# Projection matrices for each mode count
for k in n_modes_list:
    W_enc[k] = rng.normal(loc=0.0, scale=0.5, size=(k, n_features))
    print(f"W_enc[{k}] shape:", W_enc[k].shape)


n_features: 9
W_enc[6] shape: (6, 9)


In [41]:
# Apply random projection: X -> Z_k
def encode_with_projection(X, W):
    return (W @ X.T).T

Z = {}

for k in n_modes_list:
    Z[k] = encode_with_projection(X, W_enc[k])
    print(f"Z[{k}] shape:", Z[k].shape)


Z[6] shape: (4318, 6)


QRC

In [42]:


N_MODES = 6   # Set mode count per run

import pennylane as qml

# PennyLane's built-in Gaussian simulator
dev = qml.device("default.gaussian", wires=N_MODES, shots=1)

print("Ready with", N_MODES, "modes")



Ready with 6 modes


/usr/local/lib/python3.12/dist-packages/pennylane/devices/legacy_facade.py:177: PennyLaneDeprecationWarning: Setting shots on device is deprecated. Please use the `set_shots` transform on the respective QNode instead.
  warnings.warn(


In [43]:
# Hyperparameter: reservoir depth-number of Gaussian layers
DEPTH = 3  # 2 or 3

# Random reservoir parameters (fixed)
rng_res = np.random.default_rng(seed=123)

# Phase shift angles: shape (DEPTH, N_MODES)
phi = rng_res.normal(loc=0.0, scale=np.pi, size=(DEPTH, N_MODES))

# Beam splitter parameters: shape (DEPTH, N_MODES-1)
theta_bs = rng_res.normal(loc=0.0, scale=np.pi/4, size=(DEPTH, max(N_MODES-1, 1)))
phi_bs   = rng_res.normal(loc=0.0, scale=2*np.pi,   size=(DEPTH, max(N_MODES-1, 1)))

@qml.qnode(dev)
def qrc_circuit_single(z_t, meas_mode):
    """
    One QRC forward pass, for a single time step t and a single measured mode.
    z_t: array of shape (N_MODES,)
    meas_mode: integer index of the mode to measure (0 .. N_MODES-1)
    Returns: scalar <X> quadrature on that mode.
    """

    # Encoding: Displacement on each mode
    for i in range(N_MODES):
        qml.Displacement(z_t[i], 0.0, wires=i)

    # Reservoir: DEPTH Gaussian mixing layers
    for d in range(DEPTH):
        # Phase shifts
        for i in range(N_MODES):
            qml.Rotation(phi[d, i], wires=i)
        # Beamsplitters
        if N_MODES > 1:
            for i in range(N_MODES - 1):
                qml.Beamsplitter(theta_bs[d, i], phi_bs[d, i], wires=[i, i+1])

    # Readout: measure X quadrature (Quadrature operator)
    return qml.expval(qml.QuadX(meas_mode))


In [44]:
# features , projection matrix, z_shape

n_features = X.shape[1]
print("n_features:", n_features)

rng = np.random.default_rng(seed=42)

# Projection matrix for THIS mode count only
W_enc = rng.normal(loc=0.0, scale=0.5, size=(N_MODES, n_features))
print("W_enc shape:", W_enc.shape)

def encode_with_projection(X, W):
    return (W @ X.T).T  # gives shape (T, N_MODES)

# Z  (single array)
Z = encode_with_projection(X, W_enc)
print("Z shape:", Z.shape)


n_features: 9
W_enc shape: (6, 9)
Z shape: (4318, 6)


In [45]:
# Hidden State

def compute_hidden_states(Z):
    """
    Z: array of shape (T, N_MODES)
    Returns: H of shape (T, N_MODES) with hidden variables for each time step.
    """
    T = Z.shape[0]
    H = np.zeros((T, N_MODES))
    for t in range(T):
        z_t = Z[t]
        # compute hidden value for each mode
        for m in range(N_MODES):
            H[t, m] = qrc_circuit_single(z_t, m)
        if (t + 1) % 10 == 0:
            print(f"Computed hidden state {t+1}/{T}")
    return H

H = compute_hidden_states(Z)
print("H shape:", H.shape)


Computed hidden state 10/4318
Computed hidden state 20/4318
Computed hidden state 30/4318
Computed hidden state 40/4318
Computed hidden state 50/4318
Computed hidden state 60/4318
Computed hidden state 70/4318
Computed hidden state 80/4318
Computed hidden state 90/4318
Computed hidden state 100/4318
Computed hidden state 110/4318
Computed hidden state 120/4318
Computed hidden state 130/4318
Computed hidden state 140/4318
Computed hidden state 150/4318
Computed hidden state 160/4318
Computed hidden state 170/4318
Computed hidden state 180/4318
Computed hidden state 190/4318
Computed hidden state 200/4318
Computed hidden state 210/4318
Computed hidden state 220/4318
Computed hidden state 230/4318
Computed hidden state 240/4318
Computed hidden state 250/4318
Computed hidden state 260/4318
Computed hidden state 270/4318
Computed hidden state 280/4318
Computed hidden state 290/4318
Computed hidden state 300/4318
Computed hidden state 310/4318
Computed hidden state 320/4318
Computed hidden s

H_eff = H[0 : T-1]   #readout


y_eff = y[1 : T]     #target

Sanity Check!

In [46]:
#Check 1
print("X:", X.shape)
print("Z:", Z.shape)
print("H:", H.shape)
print("y:", y.shape)

print("\nAny NaN values?")
print("X:", np.isnan(X).any())
print("Z:", np.isnan(Z).any())
print("H:", np.isnan(H).any())
print("y:", np.isnan(y).any())

print("\nAny infinite values?")
print("X:", np.isinf(X).any())
print("Z:", np.isinf(Z).any())
print("H:", np.isinf(H).any())
print("y:", np.isinf(y).any())


X: (4318, 9)
Z: (4318, 6)
H: (4318, 6)
y: (4318,)

Any NaN values?
X: False
Z: False
H: False
y: False

Any infinite values?
X: False
Z: False
H: False
y: False


In [47]:
#Check 2
print("H mean:", H.mean())
print("H std:", H.std())
print("Per-mode std:", H.std(axis=0))


H mean: -0.004339732796466916
H std: 1.5537022211610103
Per-mode std: [1.14673118 1.48564175 1.81531877 1.69674181 1.33718444 1.73180228]
